In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import datasets, metrics

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("./input"))

# Any results you write to the current directory are saved as output.

['test.csv', 'trainLabels.csv', 'train.csv']


In [5]:
from urllib.request import urlopen
filename = './input/train.csv'
raw_data = open(filename, 'rt')
train = np.loadtxt(raw_data, delimiter=",")
print(train.shape)

filename = './input/trainLabels.csv'
raw_data = open(filename, 'rt')
train_labels = np.loadtxt(raw_data, delimiter=",")
print(train.shape)

filename = './input/test.csv'
raw_data = open(filename, 'rt')
test = np.loadtxt(raw_data, delimiter=",")
print(test.shape)

(1000, 40)
(1000, 40)
(9000, 40)


In [8]:

# 切分訓練集/測試集
x_train, x_test, y_train, y_test = train_test_split(train, train_labels, test_size=0.25, random_state=42)


n_estimators = [100, 200, 300]
max_depth = [1, 3, 5, 20]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(GradientBoostingClassifier(random_state=7), param_grid, scoring="accuracy", n_jobs=-1, verbose=1)

# 開始搜尋最佳參數
grid_result = grid_search.fit(x_train, y_train)



Fitting 3 folds for each of 12 candidates, totalling 36 fits


/Users/pmpmpm/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:   14.0s finished


In [9]:
# 使用最佳參數重新建立模型
clf_bestparam = GradientBoostingClassifier(max_depth=grid_result.best_params_['max_depth'],
                                           n_estimators=grid_result.best_params_['n_estimators'])

# 訓練模型
clf_bestparam.fit(x_train, y_train)

# 預測測試集
y_pred = clf_bestparam.predict(x_test)

print (metrics.classification_report(y_test, y_pred))
print(metrics.accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.92      0.88      0.90       130
         1.0       0.87      0.92      0.89       120

   micro avg       0.90      0.90      0.90       250
   macro avg       0.90      0.90      0.90       250
weighted avg       0.90      0.90      0.90       250

0.896


In [34]:
pred  = clf_bestparam.predict(test)
print(pred)
rfc_best_pred = pd.DataFrame({'Id' : np.arange(1,len(test)+1,1), 'Solution' : pred})
rfc_best_pred['Solution'] = rfc_best_pred['Solution'].astype(str).replace('\.0', '', regex=True)
print(rfc_best_pred.head())
rfc_best_pred.to_csv('Submission.csv',index=False)

[1. 0. 0. ... 1. 0. 1.]
   Id Solution
0   1        1
1   2        0
2   3        0
3   4        0
4   5        0
